In [1]:
import textract
import os
import pickle
from pathlib import Path
import numpy as np
import random
import pandas as pd
from itertools import combinations
root = Path(".")


In [3]:
my_path = root / "Pickled_files" / "Incident_Matrix"
dbfile = open(my_path, 'rb')     
incident_matrix = pickle.load(dbfile)
dbfile.close()

my_path = root / "Pickled_files" / "Shingle_id"
dbfile = open(my_path, 'rb')     
shingle_id = pickle.load(dbfile)
dbfile.close()

my_path = root / "Pickled_files" / "Shingles"
dbfile = open(my_path, 'rb')     
shingles = pickle.load(dbfile)
dbfile.close()

my_path = root / "Pickled_files" / "documents"
dbfile = open(my_path, 'rb')     
documents = pickle.load(dbfile)
dbfile.close()

In [4]:
def get_hash_functions(cnt_hsh , mod = 423212) : 
    hash_function = list()
    for i in range(cnt_hsh) : 
        a = random.randint(1 , 100)
        b = random.randint(1 , 100)
        hash_function.append((a , b , mod))
    return hash_function

In [5]:
hash_functions = get_hash_functions(50 , len(incident_matrix))
hash_functions

[(89, 93, 47147),
 (96, 3, 47147),
 (36, 54, 47147),
 (88, 28, 47147),
 (32, 78, 47147),
 (18, 29, 47147),
 (100, 35, 47147),
 (88, 79, 47147),
 (11, 34, 47147),
 (41, 14, 47147),
 (91, 67, 47147),
 (27, 87, 47147),
 (88, 35, 47147),
 (63, 54, 47147),
 (59, 74, 47147),
 (26, 76, 47147),
 (9, 95, 47147),
 (57, 64, 47147),
 (49, 69, 47147),
 (78, 51, 47147),
 (50, 40, 47147),
 (58, 69, 47147),
 (24, 74, 47147),
 (3, 38, 47147),
 (39, 42, 47147),
 (3, 54, 47147),
 (30, 25, 47147),
 (95, 77, 47147),
 (52, 76, 47147),
 (71, 1, 47147),
 (80, 26, 47147),
 (87, 57, 47147),
 (56, 58, 47147),
 (49, 98, 47147),
 (66, 3, 47147),
 (3, 67, 47147),
 (44, 69, 47147),
 (100, 63, 47147),
 (55, 60, 47147),
 (21, 1, 47147),
 (24, 16, 47147),
 (40, 59, 47147),
 (30, 57, 47147),
 (16, 79, 47147),
 (40, 31, 47147),
 (22, 49, 47147),
 (56, 93, 47147),
 (89, 54, 47147),
 (99, 26, 47147),
 (95, 40, 47147)]

In [6]:
signature_matrix = np.zeros(shape=(len(hash_functions), len(documents)))
signature_matrix.fill(10 ** 18)
signature_matrix

array([[1.e+18, 1.e+18, 1.e+18, ..., 1.e+18, 1.e+18, 1.e+18],
       [1.e+18, 1.e+18, 1.e+18, ..., 1.e+18, 1.e+18, 1.e+18],
       [1.e+18, 1.e+18, 1.e+18, ..., 1.e+18, 1.e+18, 1.e+18],
       ...,
       [1.e+18, 1.e+18, 1.e+18, ..., 1.e+18, 1.e+18, 1.e+18],
       [1.e+18, 1.e+18, 1.e+18, ..., 1.e+18, 1.e+18, 1.e+18],
       [1.e+18, 1.e+18, 1.e+18, ..., 1.e+18, 1.e+18, 1.e+18]])

In [7]:
for shingle_row in range(len(incident_matrix)) : 
    cur_hsh = 0
    for (a , b , mod) in hash_functions : 
        hsh_value = ((shingle_row + 1) * a + b) % mod
        shingle_row_val = incident_matrix[shingle_row]
        for doc_id in range(len(shingle_row_val)) : 
            if shingle_row_val[doc_id] == 1 : 
                signature_matrix[cur_hsh][doc_id] = min(signature_matrix[cur_hsh][doc_id] , hsh_value)
        cur_hsh += 1

In [8]:
print(signature_matrix)

[[32. 25. 29. ... 27. 32. 32.]
 [ 4.  2.  4. ...  4.  2.  4.]
 [ 5.  4.  5. ...  6.  4.  7.]
 ...
 [56. 10. 21. ... 12. 12. 12.]
 [ 8.  8.  8. ...  8.  2. 82.]
 [ 6.  6.  6. ... 26.  6. 67.]]


In [9]:
my_path = root / "Pickled_files" / "Signature_Matrix"
dbfile = open(my_path, 'wb')
pickle.dump(signature_matrix, dbfile) 
dbfile.close()

In [10]:
def get_candidate_pairs(b , r) : 
    candidate_pairs = set()
    cur_band_start , cur_band_end = 0 , r - 1
    for bands in range(b) :
        band_signature = dict() 
        for doc_id in range(len(documents)) : 
            matrix = tuple(signature_matrix[cur_band_start : cur_band_end + 1 , doc_id])
            if band_signature.get(matrix) == None : 
                band_signature[matrix] = [doc_id]
            else : 
                band_signature[matrix].append(doc_id)
        for docs in band_signature.items() : 
            if len(docs[1]) > 1 : 
                c_pairs = list(combinations(docs[1] , 2))
                for (a , b) in c_pairs : 
                    candidate_pairs.add((min(a , b) , max(a , b)))
        cur_band_end += r
        cur_band_start += r
    return candidate_pairs

In [11]:
candidate_pairs = get_candidate_pairs(10 , 5)

In [12]:
candidate_pairs

{(0, 165),
 (0, 286),
 (1, 71),
 (1, 241),
 (1, 426),
 (2, 130),
 (2, 158),
 (2, 170),
 (2, 348),
 (2, 475),
 (3, 12),
 (3, 123),
 (3, 194),
 (3, 453),
 (3, 462),
 (3, 483),
 (7, 84),
 (9, 320),
 (10, 188),
 (10, 228),
 (10, 483),
 (11, 101),
 (12, 35),
 (12, 328),
 (14, 257),
 (14, 441),
 (15, 280),
 (17, 73),
 (17, 120),
 (17, 155),
 (17, 230),
 (17, 276),
 (17, 391),
 (17, 497),
 (21, 401),
 (22, 71),
 (22, 479),
 (24, 37),
 (24, 95),
 (24, 118),
 (24, 190),
 (24, 378),
 (24, 411),
 (24, 458),
 (24, 482),
 (26, 42),
 (26, 191),
 (26, 445),
 (27, 114),
 (27, 162),
 (27, 475),
 (29, 38),
 (29, 440),
 (29, 447),
 (29, 486),
 (32, 122),
 (32, 365),
 (35, 328),
 (35, 431),
 (36, 402),
 (36, 452),
 (37, 378),
 (37, 414),
 (37, 458),
 (37, 482),
 (37, 486),
 (38, 440),
 (40, 156),
 (40, 241),
 (40, 438),
 (41, 356),
 (41, 421),
 (41, 430),
 (41, 451),
 (42, 191),
 (43, 379),
 (48, 57),
 (48, 72),
 (48, 181),
 (48, 216),
 (48, 221),
 (49, 190),
 (49, 461),
 (50, 260),
 (50, 407),
 (50, 423)

In [15]:
def get_similar_documents(threshold) : 
    similar_docs = set()
    col_size = len(incident_matrix)
    for doc1 in range(len(documents)) : 
        for doc2 in range(doc1 + 1 , len(documents)) : 
            matrix1 = incident_matrix[0:col_size , doc1]
            matrix2 = incident_matrix[0:col_size , doc2]
            matrix1 , matrix2 = matrix1.astype(int) , matrix2.astype(int)
            similarity = jaccard_similarity(matrix1 , matrix2)
            print(similarity)
            if similarity >= threshold : 
                similar_docs.add((doc1 , doc2 , similarity))

In [14]:
def jaccard_similarity(a , b) :
    bits = (a & b)
    dif = (a ^ b)
    return float(np.count_nonzero(bits == 1) / (np.count_nonzero(dif == 1) + np.count_nonzero(bits == 1))) 

In [16]:
similar_docs = get_similar_documents(0.5)
print(similar_docs)

0.21513892495455725
0.22939244663382594
0.20749322900993078
0.23044809351516837
0.09225874867444327
0.23808172531214528
0.2265296590378328
0.21501320256506978
0.1978911230995586
0.25919201089423516
0.2577556491765607
0.2300955414012739
0.19711407568744896
0.2148444953253196
0.22381432896064582
0.25967970808838436
0.2554684275691292
0.19846069268829028
0.144853875476493
0.20174799708667152
0.21788747346072188
0.22890805806858827
0.18806410020261558
0.27365511375717627
0.27061153750928446
0.22852445540484997
0.257186957360602
0.2205014749262537
0.2443735035913807
0.21668597914252608
0.25681716584711667
0.22962595987119147
0.17308266189197255
0.1963876146788991
0.21254443880142204
0.20023557126030625
0.20030283365779797
0.21180831347630394
0.2572550730962252
0.23731728288907997
0.26971677559912854
0.26141864870553566
0.2674037674037674
0.24207157404713414
0.23346630275036742
0.23098167210645243
0.16548536209553158
0.27054108216432865
0.21550387596899226
0.22895180948809302
0.2456850628595

In [17]:
similar_docs

In [19]:
candidate_pairs = list(candidate_pairs)
for i in range(5) : 
    print("1.)" , documents[candidate_pairs[i][0]])
    print("2.)" , documents[candidate_pairs[i][1]])
    print("----------------------------------------------------------")

1.)  want to get this briefing by email here s the good evening here s the latest 1 the declassified version of the intelligence report on russian hacking ordered by president obama found that russia s leader vladimir v putin ordered an influence campaign to sway the 2016 election he turned from seeking to denigrate hillary clinton to developing a clear preference for donald j trump the report states mr trump was briefed on the report s contents by the nation s top intelligence and law enforcement officials afterward the acknowledged the possibility that russia had hacked a variety of targets but asserted that the hacking had not affected the outcome of the election 2 at least five people were killed and eight wounded in a shooting at fort international airport in florida the gunman targeted people in the baggage claim area of a terminal used by delta and air canada and was quickly taken into custody senator bill nelson identified him as esteban santiago and said he was carrying a mili